In [50]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os
from typing import Dict, List

In [51]:
import requests

url="https://api.semanticscholar.org/CorpusID:173991054"
response = requests.get(url)

In [52]:
data = response.text
soup= BeautifulSoup(data, 'html')

In [53]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<title>[PDF] Hierarchical Decision Making by Generating and Following Natural Language Instructions | Semantic Scholar</title>
<meta content="noarchive" name="robots"/>
<!-- We have a public API for this page. Check out http://api.semanticscholar.org/ -->
<link href="https://www.semanticscholar.org/paper/Hierarchical-Decision-Making-by-Generating-and-Hu-Yarats/171bffb23613e89d60664e2078c9e807d0e523df" rel="canonical"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta charset="utf-8"/>
<meta content="39ffc7518767fb4258a6aaf807f96bfbc91f8d7e" name="s2-ui-version"/>
<meta content="We explore using latent natural language instructions as an expressive and compositional representation of complex actions for hierarchical decision making. Rather than directly selecting micro-actions, our agent first generates a latent plan in natural language, which is then executed by a separate model. We introduce a challenging real-time str

In [54]:
mydivs = soup.find_all("link", {"rel": "canonical"})
result = os.popen(f"curl {mydivs[0].attrs['href']}").read()


In [55]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1024x1400")

# download Chrome Webdriver
# https://sites.google.com/a/chromium.org/chromedriver/download
# put driver executable file in the script directory
chrome_driver = os.path.join(os.getcwd(), "chromedriver")

driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=chrome_driver)

<ipython-input-55-bf3a5068e58a>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=chrome_driver)


In [56]:
driver.get(mydivs[0].attrs['href'])
buttons = driver.find_elements_by_css_selector("button[data-heap-id='citation_box']")

for button in buttons:
    driver.execute_script("arguments[0].click();", button)
source = driver.page_source

In [57]:
new_soup = BeautifulSoup(source, 'html.parser')
citation_excerpt_sources = new_soup.find_all("div", {"class": "cl-citation__excerpts__citation"})

In [58]:
for excerpt in citation_excerpt_sources:
    print(excerpt)

<div class="cl-citation__excerpts__citation"><span class=""><span>More recently, [</span><em>Hu et al., 2019</em><span>] consider generated natural language as a representation for macro actions in a real-time strategy game environment based on [Tian et al., 2017].</span></span></div>
<div class="cl-citation__excerpts__citation"><span class=""><span>More recently, </span><em>[Hu et al., 2019]</em><span> consider generated natural language as a representation for macro actions in a real-time strategy game environment based on [Tian et al.</span></span></div>
<div class="cl-citation__excerpts__citation"><span class=""><span>LLPs have been used to construct interactive agents capable of complex reasoning (e.g. programming by demonstration) and planning over long horizons (e.g. in strategy games; </span><em>Hu et al., 2019</em><span>).</span></span></div>
<div class="cl-citation__excerpts__citation"><span class=""><span>We perform this evaluation for multiple game configurations: original,

In [74]:
def load_chrome_webdriver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1024x1400")

    # download Chrome Webdriver
    # https://sites.google.com/a/chromium.org/chromedriver/download
    # put driver executable file in the script directory
    chrome_driver = os.path.join(os.getcwd(), "chromedriver")

    driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=chrome_driver)
    return driver

def extract_excerpts(url: str) -> Dict[str, List[str]]:
    # Must Install Chrome Driver before running
    driver = load_chrome_webdriver()
    driver.get(url)
    buttons = driver.find_elements_by_css_selector("button[data-heap-id='citation_box']")
    for button in buttons:
        driver.execute_script("arguments[0].click();", button)
    new_soup = BeautifulSoup(driver.page_source, 'html.parser')
    citation_excerpt_sources = new_soup.find_all("div", {"class": "cl-citation__excerpts__citation"})
    citations = []
    for excerpt in citation_excerpt_sources:
        cite_type = excerpt.parent.previousSibling.contents[0].split()[1]
        if excerpt.parent.previousSibling.contents[0] == 'Additional Excerpts':
            cite_type = 'other'
        citations.append({'html': excerpt, 'type': cite_type})
    return citations

In [76]:
excerpts = extract_excerpts(mydivs[0].attrs['href'])

<ipython-input-74-d16a78d2c034>:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=chrome_driver)


In [77]:
excerpts

[{'html': <div class="cl-citation__excerpts__citation"><span class=""><span>More recently, [</span><em>Hu et al., 2019</em><span>] consider generated natural language as a representation for macro actions in a real-time strategy game environment based on [Tian et al., 2017].</span></span></div>, 'type': 'background'}, {'html': <div class="cl-citation__excerpts__citation"><span class=""><span>More recently, </span><em>[Hu et al., 2019]</em><span> consider generated natural language as a representation for macro actions in a real-time strategy game environment based on [Tian et al.</span></span></div>, 'type': 'background'}, {'html': <div class="cl-citation__excerpts__citation"><span class=""><span>LLPs have been used to construct interactive agents capable of complex reasoning (e.g. programming by demonstration) and planning over long horizons (e.g. in strategy games; </span><em>Hu et al., 2019</em><span>).</span></span></div>, 'type': 'methods'}, {'html': <div class="cl-citation__excer